In [ ]:
import numpy as np
with open("input_13") as f:
    input = f.read().split("\n\n")
input

In [ ]:
machines = []
for line in input:
    split = line.split("\n")
    machine = {}
    machine["A"] = np.asarray((int(split[0][12:14]), int(split[0][-2:])) )
    machine["B"] = np.asarray((int(split[1][12:14]), int(split[1][-2:])) )
    prize = split[2].split(" ")
    machine["prize"] = np.asarray((int(prize[1][2:-1]), int(prize[2][2:])) )
    machines.append(machine)
machines

In [74]:
def find_lowest_price(machine):
    # start from low A and high B
    avals = np.arange(101)
    bvals = np.arange(101,0,-1)
    #vecs = np.stack((avals,bvals),1)
    for a in avals:
        for b in bvals:
            if np.all(a * machine["A"] + b * machine["B"] == machine["prize"]):
                return (a*3 + b).item()
    return 0
        

In [ ]:
c = 0
for machine in machines:
    c += find_lowest_price(machine)
c